# Recap

First we need access to data. 
- You can use this link to add the data to your drive: https://drive.google.com/drive/folders/1pHNxZVrlcKh5usWoNC_V7gR2WdeDutjv

- Then go inside the folder **CS_for_MedStudents_data** and you will see the folder **HAM10000**.
- Right click on the **HAM10000** folder and click on the **Add to my Drive** option.

Now you can run the next cell

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Check the directory
data_dir = "/content/drive/My Drive/HAM10000"

classes = [ 'actinic keratoses', 'basal cell carcinoma', 'benign keratosis-like lesions', 
           'dermatofibroma','melanoma', 'melanocytic nevi', 'vascular lesions']

In [0]:
import torchvision.transforms as transforms

# Imagenet values
norm_mean = (0.4914, 0.4822, 0.4465)
norm_std = (0.2023, 0.1994, 0.2010)

# define the transformaitons the images go through each time it is used for training
# includes augmentation AND normalization as descirbed above
augmentation_train = transforms.Compose([
                                  # resize image to the network input size
                                  transforms.Resize((224,224)),
                                  # randomly perform a horizontal flip of the image
                                  transforms.RandomHorizontalFlip(),
                                  # rotate the image with a angle from 0 to 60 (chosen randomly)
                                  transforms.RandomRotation(degrees=60),
                                  # convert the image into a tensor so it can be processed by the GPU
                                  transforms.ToTensor(),
                                  # normalize the image with the mean and std of ImageNet
                                  transforms.Normalize(norm_mean, norm_std),
                                   ])

In [0]:
# no augmentation for the test data only resizing, conversion to tensor and normalization
augmentation_test = transforms.Compose([
                    transforms.Resize((224,224)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                    ])


In [0]:
import torchvision

# create an instance of the image folder class to load images by classes defined with the folders given
dataset = torchvision.datasets.ImageFolder(root= data_dir, transform= augmentation_train)

In [0]:
import torch
from sklearn.model_selection import train_test_split

# get the total amount of images in the dataset
num_train = len(dataset)

# create a list of indices for the whole dataset
indices = list(range(num_train))

# get the class labels from the dataset object (0-6)
class_labels = dataset.targets

# define the percentage of data that is not used for training
split_size = 0.2

# call a function of sklarn that takes care of splitting the dataset into training and validation+testing
train_indices, test_indices, class_labels_train, class_labels_test = train_test_split(indices,
                                                                                       class_labels,
                                                                                       test_size=split_size,
                                                                                       shuffle=True,
                                                                                       stratify= class_labels,
                                                                                       random_state=42)

# call a function of sklearn that splits validation+training into validation and training
train_indices, val_indices = train_test_split(train_indices,
                                               test_size=split_size,
                                               shuffle=True,
                                               stratify= class_labels_train,
                                               random_state=42)

# Creating data samplers and loaders using the indices:
SubsetRandomSampler = torch.utils.data.sampler.SubsetRandomSampler

# create instances of a torch class for picking random samples from our dataset
train_samples = SubsetRandomSampler(train_indices)
val_samples = SubsetRandomSampler(val_indices)
test_samples = SubsetRandomSampler(test_indices)

In [0]:
# define the batch size for training, val and testing
batch_size, validation_batch_size, test_batch_size = 32, 32, 32

# create and instance of a dataloader for training
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False,num_workers=1, sampler= train_samples)

# overwrite the dataset instance with the test augmentation (this is not nice code)
dataset = torchvision.datasets.ImageFolder(root= data_dir, transform=augmentation_test)
# create instances of a dataloaders for validation and testing
validation_data_loader = torch.utils.data.DataLoader(dataset, batch_size=validation_batch_size, shuffle=False, sampler=val_samples)
test_data_loader = torch.utils.data.DataLoader(dataset, batch_size=test_batch_size, shuffle=False, sampler=test_samples)

# Define a Convolutional Neural Network

Pytorch makes it very easy to define a neural network. We have layers like Convolutions, ReLU non-linearity, Maxpooling etc. directly from torch library.

In this tutorial, we use The LeNet architecture introduced by LeCun et al. in their 1998 paper, [Gradient-Based Learning Applied to Document Recognition](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf). As the name of the paper suggests, the authors’ implementation of LeNet was used primarily for OCR and character recognition in documents.

The LeNet architecture is straightforward and small, (in terms of memory footprint), making it perfect for teaching the basics of CNNs.

In [0]:
from torch import nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = len(classes)
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, (5,5), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (5,5)) 
        self.fc1   = nn.Linear(16*54*54, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, num_classes)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = LeNet()
net = net.to(device)

# Define a Loss function

Let's use a Classification Cross-Entropy loss.

$H_{y'} (y) := - \sum_{i} y_{i}' \log (y_i)$

### Median Frequency Balancing
In Exercise 6 you had seen that the number of samples per classes in the HAM10000 is not equal. 

![Class Distribution](https://github.com/IFL-CAMP/ML_for_MedStudents/blob/master/Images/class_dist.png?raw=true)

So, if we train using this data, our model will be biased towards the class  with more samples, i.e nv or melanocytic nevi. Therefore, we need to counter this class imbalance in our dataset. As a solution, we use **Median Frequency Balancing**.

In [9]:
# Median Frequency Balancing

import numpy as np

# get the class labels of each image
class_labels = dataset.targets
# empty array for counting instance of each class
count_labels = np.zeros(len(classes))
# empty array for weights of each class
class_weights = np.zeros(len(classes))

# populate the count array
for l in class_labels:
  count_labels[l] += 1

# get median count
median_freq = np.median(count_labels)

# calculate the weigths
for i in range(len(classes)):
  class_weights[i] = median_freq/count_labels[i]

# print the weights
for i in range(len(classes)):
    print(classes[i],":", class_weights[i])

actinic keratoses : 1.5718654434250765
basal cell carcinoma : 1.0
benign keratosis-like lesions : 0.467697907188353
dermatofibroma : 4.469565217391304
melanoma : 0.4618149146451033
melanocytic nevi : 0.07665920954511558
vascular lesions : 3.619718309859155


Now we define the loss function with the weights

In [0]:
class_weights = torch.FloatTensor(class_weights).to(device)
criterion = nn.CrossEntropyLoss(weight = class_weights)

# Define the Optimizer

The most common and effective Optimizer currently used is **Adam: Adaptive Moments**. You can look [here](https://arxiv.org/abs/1412.6980) for more information.


In [11]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=1e-5)
print(net)

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=46656, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=7, bias=True)
)


# Training

In [15]:
num_epochs = 5
accuracy = []
val_accuracy = []
losses = []
val_losses = []


for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_data_loader):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # set the parameter gradients to zero
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #compute accuracy
        _, predicted = torch.max(outputs, 1)
        running_loss += loss.item()

    
    running_loss /= len(train_data_loader)

    print('Epoch: {}'.format(epoch+1))
    print('Loss: {}' .format(running_loss))

print('Finished Training')

Epoch: 1
Loss: 1.5936264689288921
Epoch: 2
Loss: 1.571835981079595
Epoch: 3
Loss: 1.5633033460645533
Epoch: 4
Loss: 1.5391058927744776
Epoch: 5
Loss: 1.5206250545397326
Finished Training


In [18]:
from torchsummary import summary
summary(net, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 224, 224]             456
            Conv2d-2         [-1, 16, 108, 108]           2,416
            Linear-3                  [-1, 120]       5,598,840
            Linear-4                   [-1, 84]          10,164
            Linear-5                    [-1, 7]             595
Total params: 5,612,471
Trainable params: 5,612,471
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 3.72
Params size (MB): 21.41
Estimated Total Size (MB): 25.71
----------------------------------------------------------------
